In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"
data = requests.get(standings_url)

In [7]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [8]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [9]:
team_urls

['https://fbref.com/en/squads/53a2f082/2023-2024/Real-Madrid-Stats',
 'https://fbref.com/en/squads/206d90db/2023-2024/Barcelona-Stats',
 'https://fbref.com/en/squads/9024a00a/2023-2024/Girona-Stats',
 'https://fbref.com/en/squads/db3b9613/2023-2024/Atletico-Madrid-Stats',
 'https://fbref.com/en/squads/2b390eca/Athletic-Club-Stats',
 'https://fbref.com/en/squads/e31d1cd9/Real-Sociedad-Stats',
 'https://fbref.com/en/squads/fc536746/Real-Betis-Stats',
 'https://fbref.com/en/squads/2a8183b3/Villarreal-Stats',
 'https://fbref.com/en/squads/dcc91a7b/Valencia-Stats',
 'https://fbref.com/en/squads/8d6fd021/Alaves-Stats',
 'https://fbref.com/en/squads/03c57e2b/Osasuna-Stats',
 'https://fbref.com/en/squads/7848bd64/Getafe-Stats',
 'https://fbref.com/en/squads/f25da7fb/Celta-Vigo-Stats',
 'https://fbref.com/en/squads/ad2be733/Sevilla-Stats',
 'https://fbref.com/en/squads/2aa12281/Mallorca-Stats',
 'https://fbref.com/en/squads/0049d422/Las-Palmas-Stats',
 'https://fbref.com/en/squads/98e8af82/Rayo

In [10]:
data = requests.get(team_urls[0])

In [11]:
import pandas as pd

matches = pd.read_html(data.text, match="Scores & Fixtures")  

In [12]:
matches[0].head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2023-08-12,21:30,La Liga,Matchweek 1,Sat,Away,W,2,0,Athletic Club,0.9,0.4,54,48927,Dani Carvajal,4-1-2-1-2◆,Jesús Gil,Match Report,NaN
1,2023-08-19,19:30,La Liga,Matchweek 2,Sat,Away,W,3,1,Almería,2.0,1.3,57,17561,Dani Carvajal,4-3-1-2,José Sánchez,Match Report,NaN
2,2023-08-25,21:30,La Liga,Matchweek 3,Fri,Away,W,1,0,Celta Vigo,1.4,1.2,63,23057,Dani Carvajal,4-1-2-1-2◆,Isidro Díaz de Mera,Match Report,NaN
3,2023-09-02,16:15,La Liga,Matchweek 4,Sat,Home,W,2,1,Getafe,2.8,0.4,76,66747,Luka Modrić,4-3-1-2,Mario Melero,Match Report,NaN
4,2023-09-17,21:00,La Liga,Matchweek 5,Sun,Home,W,2,1,Real Sociedad,2.0,1.6,52,70092,Dani Carvajal,4-1-2-1-2◆,César Soto,Match Report,NaN


In [13]:
soup = BeautifulSoup(data.text)

In [14]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [15]:
links

['/en/squads/53a2f082/2023-2024/matchlogs/all_comps/shooting/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2023-2024/matchlogs/all_comps/shooting/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2023-2024/matchlogs/all_comps/shooting/Real-Madrid-Match-Logs-All-Competitions',
 '/en/squads/53a2f082/2023-2024/matchlogs/all_comps/shooting/Real-Madrid-Match-Logs-All-Competitions']

In [16]:
data = requests.get(f"https://fbref.com{links[0]}")

In [17]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [19]:
shooting.columns = shooting.columns.droplevel()

In [21]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [22]:
shooting.shape

(56, 26)

In [23]:
years = list(range(2024, 2020, -1))

In [24]:
years

[2024, 2023, 2022, 2021]

In [25]:
all_matches = []

In [26]:
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [27]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "La Liga"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)

In [28]:
match_df = pd.concat(all_matches)

In [29]:
match_df.columns = [c.lower() for c in match_df.columns]

In [30]:
match_df.to_csv("morematches.csv")

In [31]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-12,21:30,La Liga,Matchweek 1,Sat,Away,W,2,0,Athletic Club,...,Match Report,NaN,14.0,8.0,16.0,2.0,0,0,2024,Real Madrid
1,2023-08-19,19:30,La Liga,Matchweek 2,Sat,Away,W,3,1,Almería,...,Match Report,NaN,25.0,9.0,17.0,1.0,0,0,2024,Real Madrid
2,2023-08-25,21:30,La Liga,Matchweek 3,Fri,Away,W,1,0,Celta Vigo,...,Match Report,NaN,9.0,2.0,19.4,0.0,0,1,2024,Real Madrid
3,2023-09-02,16:15,La Liga,Matchweek 4,Sat,Home,W,2,1,Getafe,...,Match Report,NaN,26.0,12.0,17.7,0.0,0,0,2024,Real Madrid
4,2023-09-17,21:00,La Liga,Matchweek 5,Sun,Home,W,2,1,Real Sociedad,...,Match Report,NaN,17.0,8.0,15.9,1.0,0,0,2024,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2021-05-01,14:00,La Liga,Matchweek 34,Sat,Home,W,3,0,Alavés,...,Match Report,NaN,15.0,4.0,16.1,0.0,0,0,2021,Eibar
37,2021-05-09,14:00,La Liga,Matchweek 35,Sun,Away,W,1,0,Getafe,...,Match Report,NaN,6.0,2.0,20.6,0.0,1,1,2021,Eibar
38,2021-05-13,20:00,La Liga,Matchweek 36,Thu,Home,D,1,1,Betis,...,Match Report,NaN,14.0,4.0,16.3,0.0,0,0,2021,Eibar
39,2021-05-16,18:30,La Liga,Matchweek 37,Sun,Away,L,1,4,Valencia,...,Match Report,NaN,20.0,8.0,15.0,0.0,0,0,2021,Eibar


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [5]:
standings_url = "https://fbref.com/en/comps/12/2024-2025/2024-2025-La-Liga-Stats"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text, 'html.parser')

In [6]:
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get('href') for l in links if '/squads/' in l.get('href')]
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
def scrape_team_matches(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.text, 'html.parser')
    matches = pd.read_html(data.text, match="Scores & Fixtures", flavor='html5lib')[0]
    return matches

In [10]:
all_matches = []
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    print(f"Scraping matches for {team_name}")
    team_matches = scrape_team_matches(team_url)
    team_matches['Team'] = team_name
    all_matches.append(team_matches)
    time.sleep(10)


Scraping matches for Real Madrid
Scraping matches for Barcelona
Scraping matches for Valencia
Scraping matches for Atletico Madrid
Scraping matches for Osasuna
Scraping matches for Mallorca
Scraping matches for Sevilla
Scraping matches for Athletic Club
Scraping matches for Getafe
Scraping matches for Real Sociedad
Scraping matches for Real Betis
Scraping matches for Espanyol
Scraping matches for Rayo Vallecano
Scraping matches for Celta Vigo
Scraping matches for Valladolid
Scraping matches for Girona
Scraping matches for Villarreal
Scraping matches for Las Palmas
Scraping matches for Alaves
Scraping matches for Leganes


In [11]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

In [12]:
match_df.to_csv("LaLiga2024-2025.csv", index=False)